# Multi hot + Frequnecy 기반 encoder
data
*   data/multi_freq_based_encoding_X.csv
*   data/multi_freq_based_encoding_X_test.csv

# Import library

In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder
import dill as pickle  

# Load Data

In [2]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")

# Data Preprocessing

In [3]:
# SUBCLASS 가 범주형이기 때문에 LabelEncoder 사용
le_subclass = LabelEncoder()
train['SUBCLASS'] = le_subclass.fit_transform(train['SUBCLASS'])

# 변환된 레이블 확인
for i, label in enumerate(le_subclass.classes_):
    print(f"원래 레이블: {label}, 변환된 숫자: {i}")

원래 레이블: ACC, 변환된 숫자: 0
원래 레이블: BLCA, 변환된 숫자: 1
원래 레이블: BRCA, 변환된 숫자: 2
원래 레이블: CESC, 변환된 숫자: 3
원래 레이블: COAD, 변환된 숫자: 4
원래 레이블: DLBC, 변환된 숫자: 5
원래 레이블: GBMLGG, 변환된 숫자: 6
원래 레이블: HNSC, 변환된 숫자: 7
원래 레이블: KIPAN, 변환된 숫자: 8
원래 레이블: KIRC, 변환된 숫자: 9
원래 레이블: LAML, 변환된 숫자: 10
원래 레이블: LGG, 변환된 숫자: 11
원래 레이블: LIHC, 변환된 숫자: 12
원래 레이블: LUAD, 변환된 숫자: 13
원래 레이블: LUSC, 변환된 숫자: 14
원래 레이블: OV, 변환된 숫자: 15
원래 레이블: PAAD, 변환된 숫자: 16
원래 레이블: PCPG, 변환된 숫자: 17
원래 레이블: PRAD, 변환된 숫자: 18
원래 레이블: SARC, 변환된 숫자: 19
원래 레이블: SKCM, 변환된 숫자: 20
원래 레이블: STES, 변환된 숫자: 21
원래 레이블: TGCT, 변환된 숫자: 22
원래 레이블: THCA, 변환된 숫자: 23
원래 레이블: THYM, 변환된 숫자: 24
원래 레이블: UCEC, 변환된 숫자: 25


In [4]:
## x 의 경우도 범주형으로 구성되어 있어, 알맞은 인코딩 필요
X = train.drop(columns=['SUBCLASS', 'ID'])
y_subclass = train['SUBCLASS']

* 이 코드는 다음과 같이 작동합니다:
1. 데이터를 로드합니다.
2. 샘플의 변이 데이터를 전처리하여 "유전자_변이" 형식의 문자열 리스트로 변환합니다.
3. Word2Vec 모델을 사용하여 변이 임베딩을 학습합니다. 여기서는 10차원 벡터를 사용했지만, 이는 조정 가능합니다.
4. 각 변이에 대한 임베딩을 가져오는 함수와 각 샘플의 임베딩을 계산하는 함수를 정의합니다. 샘플 임베딩은 해당 샘플의 모든 변이 임베딩의 평균으로 계산됩니다.
5. 모든 샘플에 대한 임베딩을 생성합니다.
6. SUBCLASS를 숫자로 인코딩합니다.
7. 최종 데이터셋을 생성합니다. 이 데이터셋은 샘플 ID, 임베딩 벡터, 인코딩된 SUBCLASS를 포함합니다.
8. 이 방법의 장점은 다음과 같습니다:
9. 변이 간의 의미적 관계를 포착할 수 있습니다.
10. 고차원 데이터를 저차원으로 축소할 수 있습니다.
11. 새로운 또는 희귀한 변이에 대해서도 의미 있는 표현을 제공할 수 있습니다.
* 단점은 다음과 같습니다:
충분한 데이터가 없으면 임베딩의 품질이 떨어질 수 있습니다.
하이퍼파라미터 튜닝(예: vector_size, window 등)이 필요할 수 있습니다.
이 인코딩 방법을 사용한 후, 결과 데이터셋을 머신러닝 모델의 입력으로 사용할 수 있습니다.

In [5]:
# 변이 사전 및 빈도 계산 함수
def create_variant_dict_and_frequencies(df):
    variant_dict = {}
    freq_dict = defaultdict(lambda: defaultdict(float))
    total_samples = len(df)
    
    for column in df.columns:  # ID와 SUBCLASS를 제외한 열
        unique_variants = df[column].unique()
        variant_dict[column] = {variant: i for i, variant in enumerate(unique_variants)}
        
        value_counts = df[column].value_counts()
        for variant, count in value_counts.items():
            freq_dict[column][variant] = count / total_samples
    
    return variant_dict, freq_dict

In [6]:
# 샘플 인코딩 함수
def encode_sample(row, variant_dict, freq_dict):
    encoded = []
    for column, value in row.items():
        if column not in ['ID', 'SUBCLASS']:
            gene_encoding = [0.0] * len(variant_dict.get(column, []))
            
            # value가 문자열일 때만 split() 적용
            if isinstance(value, str):
                mutations = value.split()
                for mutation in mutations:
                    if mutation in variant_dict.get(column, {}):
                        gene_encoding[variant_dict[column][mutation]] = freq_dict[column][mutation]
            
            # 문자열이 아닌 경우 건너뜀
            encoded.extend(gene_encoding)
    return encoded

In [7]:
# 변이 사전 및 빈도 계산
variant_dict, freq_dict = create_variant_dict_and_frequencies(X)

In [8]:
%%time
# 데이터 인코딩
encoded_data = X.apply(lambda row: encode_sample(row, variant_dict, freq_dict), axis=1)

CPU times: user 21.9 s, sys: 2.08 s, total: 23.9 s
Wall time: 26 s


In [9]:
encoded_data.shape

(6201,)

In [10]:
encoded_data

0       [0.9761328817932592, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1       [0.9761328817932592, 0.0, 0.0, 0.0, 0.0, 0.0, ...
2       [0.0, 0.00016126431220770844, 0.0, 0.0, 0.0, 0...
3       [0.9761328817932592, 0.0, 0.0, 0.0, 0.0, 0.0, ...
4       [0.9761328817932592, 0.0, 0.0, 0.0, 0.0, 0.0, ...
                              ...                        
6196    [0.9761328817932592, 0.0, 0.0, 0.0, 0.0, 0.0, ...
6197    [0.9761328817932592, 0.0, 0.0, 0.0, 0.0, 0.0, ...
6198    [0.9761328817932592, 0.0, 0.0, 0.0, 0.0, 0.0, ...
6199    [0.9761328817932592, 0.0, 0.0, 0.0, 0.0, 0.0, ...
6200    [0.9761328817932592, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Length: 6201, dtype: object

In [11]:
# 인코딩된 데이터를 DataFrame으로 변환
column_names = [f"{gene}_{variant}" for gene, variants in variant_dict.items() for variant in variants]
encoded_X = pd.DataFrame(encoded_data.tolist(), columns=column_names)

In [12]:
# 변이 사전 및 빈도 사전 저장 (테스트 데이터 인코딩을 위해)
with open('variant_and_freq_dict.pkl', 'wb') as f:
    pickle.dump((variant_dict, freq_dict), f)

In [13]:
X_test = test.drop(["ID"], axis = 1)

In [14]:
# 변이 사전 및 빈도 사전 로드
with open('variant_and_freq_dict.pkl', 'rb') as f:
    variant_dict, freq_dict = pickle.load(f)

In [15]:
# 테스트 데이터 인코딩
test_encoded = X_test.apply(lambda row: encode_sample(row, variant_dict, freq_dict), axis=1)

In [16]:
# 인코딩된 테스트 데이터를 DataFrame으로 변환
column_names = [f"{gene}_{variant}" for gene, variants in variant_dict.items() for variant in variants]
test_encoded_df = pd.DataFrame(test_encoded.tolist(), columns=column_names)

Save in data

* encoded_data.to_csv(f'../data/multi_freq_based_encoding_X.csv', encoding='UTF-8-sig', index=False)
* test_encoded_df.to_csv(f'../data/multi_freq_based_encoding_X_test.csv', encoding='UTF-8-sig', index=False)

In [17]:
from pycaret.classification import *

In [18]:
# 예제 데이터셋 로드
X = encoded_X
y = pd.DataFrame(y_subclass)

In [19]:
X.shape

(6201, 206989)

In [20]:
X = X.loc[:, (X != 0).any(axis=0)]

In [21]:
X.shape

(6201, 185171)

In [1]:
# 환경 설정
setup_clf = setup(data=X, target=y["SUBCLASS"], session_id=123,
                  normalize=True,  # 정규화 (Min-Max 스케일링)
                  transformation=True,  # 분포 변환 (예: Box-Cox)
                  normalize_method='minmax')  # 정규화 방법 지정

NameError: name 'setup' is not defined